<a href="https://colab.research.google.com/github/dguevaraapaza/H1N1-Seasonal-Flu-Vaccine/blob/main/H1N1_and_seasonal_flu_vaccines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vaccination Prediction


> **Goal:** Predict how likely individuals are to receive their H1N1 and seasonal flu vaccines. In specific, the target is to predict two probabilities: one for h1n1_vaccine and one for seasonal_vaccine.

**This is a multi-label classification problem.**

In [1]:
# !pip install -q pandas
# !pip install -q numpy
# !pip install -q matplotlib
# !pip install -q scikit-learn
# !pip install -q seaborn
# !pip install -q missingno
# !pip install -q fancyimpute

In [2]:
# # Installing the required packages
# !pip install scikit-multilearn

# Libraries for Problem Transformation
# from skmultilearn.problem_transform import BinaryRelevance
# from skmultilearn.problem_transform import ClassifierChain
# from skmultilearn.problem_transform import LabelPowerset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA

# Libraries for model training
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score

# Libraries for algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Impute binary data
from fancyimpute import IterativeImputer

# Libraries for testing
from sklearn.metrics import accuracy_score , classification_report

# Removing warnings
import warnings
warnings.filterwarnings('ignore')

#Check missing values
import missingno as msno

# Storing plots
%matplotlib inline

In [3]:
# Load data
vac = pd.read_csv('data/fixed_H1N1_Flu_Vaccines.csv', index_col='respondent_id', delimiter=',')
display(vac.head())
print(f'Dataset strcuture: {vac.shape}')

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0,0,0,0,0,1,1,0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,0,0
1,3.0,2.0,0,1,0,1,0,1,1,0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,0,1
2,1.0,1.0,0,1,0,0,0,0,0,0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,0,0
3,1.0,1.0,0,1,0,1,1,0,0,0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,0,1
4,2.0,1.0,0,1,0,1,1,0,1,0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,0,0


Dataset strcuture: (25098, 34)


In [ ]:
# group variables for imputation
h1n1_concern = ['h1n1_concern', 'h1n1_knowledge'] # categorical
behaviour = ['behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 
             'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 
             'behavioral_touch_face'] #binary
recomendation = ['doctor_recc_h1n1', 'doctor_recc_seasonal'] #binary
health_condition = ['chronic_med_condition', 'child_under_6_months', 'health_worker'] #, 'health_insurance'] #Binary
opinion = ['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
           'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
           'opinion_seas_risk', 'opinion_seas_sick_from_vacc'] # categorical
demographics = ['age_group','education', 'race', 'sex', 'income_poverty',
                'marital_status', 'rent_or_own', 'employment_status'] #, 'employment_industry', 'employment_occupation'] # categorical
household_info = ['hhs_geo_region', 'census_msa', 'household_adults',
                  'household_children'] # categorical
predictors = ['h1n1_vaccine', 'seasonal_vaccine'] # predictor variables

# variable to impute
cat_variable = h1n1_concern + opinion + demographics + household_info
bin_variable = behaviour + recomendation + health_condition

## **1. Data preprocessing**

In [ ]:
# Prepare datasets
labels = vac[predictors] # (26707, 2)
features = vac.drop(predictors, axis=1) # (26707, 35)

In [7]:
## Encoding the objects
enc = LabelEncoder() 
for col in features[cat_variable]:
	features[col] = enc.fit_transform(features[col])


# FEATURE SCALING

## Using Min Max scaler

In [ ]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# FEATURE SELECTION

## Independent and target variables

In [ ]:
X = features
y = labels

## Applying SelectKBest class

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k=30)

In [ ]:
fit = bestfeatures.fit(X,y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']

print(featureScores.nlargest(25,'Score'))

In [ ]:
features_kbest = features.iloc[:,[9,10,19,16,13,11,21,18,15,34,0,33,32,8,12,1,24,5,26,17,6,3,7,2,4]]

## Feature importance using ExtraTrees Classifier

In [ ]:
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_)
plt.figure(figsize=(10,10))
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh')
plt.show()

In [ ]:
features_tree = features.loc[:,['opinion_seas_risk','doctor_recc_seasonal','opinion_seas_vacc_effective',
                                 'doctor_recc_h1n1','age_group','opinion_h1n1_risk','hhs_geo_region',
                                 'opinion_h1n1_vacc_effective','h1n1_concern','opinion_seas_sick_from_vacc',
                                 'opinion_h1n1_sick_from_vacc','census_msa','education','income_poverty',
                                 'h1n1_knowledge','household_adults','employment_occupation',
                                 'employment_industry','household_children','sex','race','marital_status',
                                 'rent_or_own','behavioral_outside_home','behavioral_large_gatherings']]

## Correlation Matrix using Heatmap

In [ ]:
features.corrwith(labels.h1n1_vaccine)

In [ ]:
features.corrwith(labels.seasonal_vaccine)

In [ ]:
#get correlations of each features in dataset
corrmat = features.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(features[top_corr_features].corr(),annot=True,cmap="Blues")

In [ ]:
features.drop(['opinion_seas_sick_from_vacc','hhs_geo_region','census_msa',
               'household_adults','household_children','behavioral_antiviral_meds',
               'education'], axis = 1, inplace = True)

# MODEL

## Defining independent and dependent variables

In [ ]:
X = features
y = labels

In [ ]:
X

In [ ]:
y

## Splitting the data into training and testing data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=23)

## Using Binary Relevance - Naive Bayes

In [ ]:
# Gaussian Naive Bayes

# Initializing

model_BR_GNB = BinaryRelevance(GaussianNB())

# Training

model_BR_GNB.fit(x_train,y_train)

# Predicting

y_pred_BR_GNB = model_BR_GNB.predict(x_test)

# Testing

BR_GNB = accuracy_score(y_test,y_pred_BR_GNB)
BR_GNB

## Using Binary Relevance - Decision Tree

In [ ]:
# Initializing

model_BR_DT = BinaryRelevance(DecisionTreeClassifier())

# Training

model_BR_DT.fit(x_train,y_train)

# Predicting

y_pred_BR_DT = model_BR_DT.predict(x_test)

# Testing

BR_DT = accuracy_score(y_test,y_pred_BR_DT)
BR_DT

## Using Binary Relevance - Logistic Regression

In [ ]:
# Initializing

model_BR_LR = BinaryRelevance(LogisticRegression())

# Training

model_BR_LR.fit(x_train,y_train)

# Predicting

y_pred_BR_LR = model_BR_LR.predict(x_test)

# Testing

BR_LR = accuracy_score(y_test,y_pred_BR_LR)
BR_LR

## Using Classifier Chains - Naive Bayes

In [ ]:
# Gaussian Naive Bayes

# Initializing

model_CC_GNB = ClassifierChain(GaussianNB())

# Training

model_CC_GNB.fit(x_train,y_train)

# Predicting

y_pred_CC_GNB = model_CC_GNB.predict(x_test)

# Testing

CC_GNB = accuracy_score(y_test,y_pred_CC_GNB)
CC_GNB

## Using Classifier Chains - Decision Tree

In [ ]:
# Initializing

model_CC_DT = ClassifierChain(DecisionTreeClassifier())

# Training

model_CC_DT.fit(x_train,y_train)

# Predicting

y_pred_CC_DT = model_CC_DT.predict(x_test)

# Testing

CC_DT = accuracy_score(y_test,y_pred_CC_DT)
CC_DT

## Using Classifier Chains - Logistic Regression

In [ ]:
# Initializing

model_CC_LR = ClassifierChain(LogisticRegression())

# Training

model_CC_LR.fit(x_train,y_train)

# Predicting

y_pred_CC_LR = model_CC_LR.predict(x_test)

# Testing

CC_LR = accuracy_score(y_test,y_pred_CC_LR)
CC_LR

## Using Label Powerset - Naive Bayes

In [ ]:
# Gaussian Naive Bayes

# Initializing

model_LP_GNB = LabelPowerset(GaussianNB())

# Training

model_LP_GNB.fit(x_train,y_train)

# Predicting

y_pred_LP_GNB = model_LP_GNB.predict(x_test)

# Testing

LP_GNB = accuracy_score(y_test,y_pred_LP_GNB)
LP_GNB

## Using Label Powerset - Decision Tree

In [ ]:
# Initializing

model_LP_DT = LabelPowerset(DecisionTreeClassifier())

# Training

model_LP_DT.fit(x_train,y_train)

# Predicting

y_pred_LP_DT = model_LP_DT.predict(x_test)

# Testing

LP_DT = accuracy_score(y_test,y_pred_LP_DT)
LP_DT

## Random Forest

In [ ]:
# Initializing

model_RF = RandomForestClassifier()

# Training

model_RF.fit(x_train,y_train)

# Predicting

y_pred_RF = model_RF.predict(x_test)

# Testing
RF = accuracy_score(y_test,y_pred_RF)
RF

# MODEL EVALUATION

## Accuracy score comparison chart

In [ ]:
Accuracy_Score = [BR_GNB,BR_DT,BR_LR,CC_GNB,CC_DT,CC_LR,LP_GNB,LP_DT,RF]
Models = ['BR Naive Bayes', 'BR Decision Tree' , 'BR Logistic Regression', 'CC Naive Bayes',
          'CC Decision Tree', 'CC Logistic Regression', 'LP Naive Bayes', 'LP Decision Tree','Random Forest']

In [ ]:
sns.barplot(Accuracy_Score, Models, color="m")
plt.xlabel('Accuracy Score')
plt.title('Accuracy Score')
plt.show()

#### RANDOM FOREST HAS THE HIGHEST ACCURACY SCORE